In [1]:
%load_ext line_profiler
from benchmark import benchmark
from collections import defaultdict

# Problem

In this assignment you will implement one or more algorithms for the 2SAT problem.
Here are 6 different 2SAT instances:
- 2sat1.txt
- 2sat2.txt
- 2sat3.txt
- 2sat4.txt
- 2sat5.txt
- 2sat6.txt

The file format is as follows.
In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file.
Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "$-$" sign denoting logical "not".
For example, the second line of the first data file is "-16808 75250", which indicates the clause $\neg x_{16808} \lor x_{75250}$.

Your task is to determine which of the 6 instances are satisfiable, and which are unsatisfiable.
In the box below, enter a 6-bit string, where the $i$-th bit should be 1 if the $i$-th instance is satisfiable, and 0 otherwise.
For example, if you think that the first 3 instances are satisfiable and the last 3 are not, then you should enter the string 111000 in the box below.

*DISCUSSION*: This assignment is deliberately open-ended, and you can implement whichever 2SAT algorithm you want.
For example, 2SAT reduces to computing the strongly connected components of a suitable graph (with two vertices per variable and two directed edges per clause, you should think through the details).
This might be an especially attractive option for those of you who coded up an SCC algorithm in Part 2 of this specialization.
Alternatively, you can use Papadimitriou's randomized local search algorithm.
(The algorithm from lecture is probably too slow as stated, so you might want to make one or more simple modifications to it --- even if this means breaking the analysis given in lecture --- to ensure that it runs in a reasonable amount of time.)
A third approach is via backtracking.
In lecture we mentioned this approach only in passing; see Chapter 9 of the Dasgupta-Papadimitriou-Vazirani book, for example, for more details.

## [2-SAT Problem](https://en.wikipedia.org/wiki/2-satisfiability)

A 2-satisfiability problem may be described using a Boolean expression with a special restricted form.
It is a conjunction (a Boolean and operation) of clauses, where each clause is a disjunction (a Boolean or operation) of two variables or negated variables. 

### Input
1. $n$ Boolean variables $x_1, x_2, \dots, x_n$.
2. $m$ clauses of 2 literals each. A "literal" is equivalent to $x_i$ or $\neg x_i$.

### Output
Return "Yes" if there is an assignment that simultaneously satisfies every clause, "No" otherwise.

### Example

*Given*: $(x_1 \lor x_2) \land (\neg x_1 \lor x_3) \land (x_3 \lor x_4) \land (\neg x_2 \lor \neg x_4)$.

*Return*: "Yes", via for example, $x_1 = x_3 = \text{TRUE}$ and $x_2 = x_4 = \text{FALSE}$.

## [2-SAT Reduced to Strongly Connected Components](http://www.geeksforgeeks.org/2-satisfiability-2-sat-problem/)

**Key Idea**: $(A \lor B) = \text{TRUE} \Leftrightarrow (\neg A \Rightarrow B) \land (\neg B \Rightarrow A)$.

We can then represent $(A \lor B)$ as two edges $\neg A \rightarrow B$ and $\neg B \rightarrow A$.<br>
Thus if two literals $A$ and $B$ belongs to the same strongly connected components (SCC), the truth of $A$ automatically implies the truth $B$.<br>
A contradiction will be encontered when $A$ and $\neg A$ belong to the same SCC, which indicates a "No" of the 2-SAT instance.

## [SCC via Kosaraju's Algorithm](https://en.wikipedia.org/wiki/Kosaraju%27s_algorithm)


### Wikipedia
```
1. For each vertex u of the graph, mark u as unvisited. Let L be empty.
2. For each vertex u of the graph do Visit(u), where Visit(u) is the recursive subroutine:
    If u is unvisited then:
        Mark u as visited.
        For each out-neighbour v of u, do Visit(v).
        Prepend u to L.
    Otherwise do nothing. 
3. For each element u of L in order, do Assign(u,u) where Assign(u,root) is the recursive subroutine:
    If u has not been assigned to a component then:
        Assign u as belonging to the component whose root is root.
        For each in-neighbour v of u, do Assign(v,root).
    Otherwise do nothing.
```

### Class

```
def Kosaraju(G):
  1. Compute reverse graph Grev
  2. a. Run depth-first search (DFS) loop (DFS_Loop) on Grev
     b. Record finishing time on each node
  3. Run DFS_Loop on G starting from nodes with large finishing time
  4. SCCs are the nodes with the same leaders
```

```python
def DFS_Loop(G):
  Global t # count finishing time
  Global s # leader of the SCC
  for i in range(n, 0, -1):
    if i not in explored:
      s = i
      DFS(G, i)
```

```python
def DFS(G, i):
  explored.add(i)
  leader[i] = s
  for edge(i, j) in G:
    if j not in explored:
      DFS(G, j)
  t += 1
  finishing_time[i] = t
```

In [2]:
# This cell implements the Kosaraju's algorithm

DEBUG = 3

def dfs_visit(u, G, V, L):
    """ DFS for step 1 of Kosaraju.
    u -- the starting vertex of G
    G -- the original graph
    V -- a set of unvisited vertices
    L -- the ordering list
    """
    
    if u in V:
        V.remove(u)
        for v in G[u]:
            dfs_visit(v, G, V, L)
        L.append(u)
    
    return

def dfs_assign(u, root, Grev, V, S):
    """ DFS for step 2 of Kosaraju.
    u -- the starting vertex of Grev
    root -- the root for vertex u
    Grev -- the reversed graph of original graph
    V -- a set of unvisited vertices
    S -- the final SCC {root: [followers]}
    """
    
    if u in V:
        V.remove(u)
        S[root].append(u)
        for v in Grev[u]:
            dfs_assign(v, root, Grev, V, S)
    
    return

def reverse_graph(G):
    """ Reverse the edge directions of graph G and return the reversed graph. """
    
    Grev = defaultdict(list)
    
    for u, v_list in G.iteritems():
        for v in v_list:
            Grev[v].append(u)
    
    return Grev

def print_defaultdict(G):
    """ Print a defaultdict G. """
    out = ""
    for u, v_list in G.iteritems():
        out += "{0}: {1}, ".format(u, str(v_list))
    print out[:-2] + "\n"
    return

def scc(G, Grev, V):
    """ Kosaraju's two-pass algorithm. """
    
    # copy the set of vertices
    Vcopy = V.copy()
    
    # final SCC map {root: [followers]}
    S = defaultdict(list)
    
    # ordering list
    L = []
    
    # first DFS
    for u in G.keys():
        dfs_visit(u, G, V, L)
    
    if DEBUG > 1:
        print "\nOrdering list:"
        print L, "\n"
    
#     # reverse the graph
#     Grev = reverse_graph(G)
    
    # second DFS
    while L:
        u = L.pop()
        dfs_assign(u, u, Grev, Vcopy, S)
    
    if DEBUG > 1:
        print "Strongly connected components:"
        print_defaultdict(S)
    
    return S

In [3]:
# This cell tests the new implementation of Kosaraju.

def read_test(filename):
    G = defaultdict(list)
    Grev = defaultdict(list)
    V = set()
    
    for line in open(filename, 'r'):
        u, v = map(int, line.split())
        V.add(u)
        V.add(v)
        G[u].append(v)
        Grev[v].append(u)
    
    if DEBUG > 2:
        print "\nGraph from file {0}".format(filename)
        print_defaultdict(G)
    
    return G, Grev, V

DEBUG = 3
# {1: [1], 4: [2, 4, 5], 6: [3, 6], 12: [7, 8, 9, 10, 11, 12]}
G, Grev, V = read_test("../../../part1/problem_set/5_scc/test2.txt")
S = scc(G, Grev, V)


Graph from file ../../../part1/problem_set/5_scc/test2.txt
1: [2], 2: [3, 4, 5], 3: [6], 4: [5, 7], 5: [2, 6, 7], 6: [3, 8], 7: [8, 10], 8: [7], 9: [7], 10: [9, 11], 11: [12], 12: [10]


Ordering list:
[9, 12, 11, 10, 7, 8, 6, 3, 5, 4, 2, 1] 

Strongly connected components:
8: [8, 7, 9, 10, 12, 11], 1: [1], 2: [2, 5, 4], 3: [3, 6]



In [4]:
# This cell uses SCC to solve 2-SAT

def read_2sat_clauses(filename):
    """ Read the 2-SAT clauses from a file.
    filename -- the file name
    
    return: a graph G {head: [tails]} and the set of vertices V.
    """
    
    G = defaultdict(list)
    Grev = defaultdict(list)
    V = set()
    n = None
    
    for i, line in enumerate(open(filename, 'r')):
        if i == 0:
            n = int(line.split()[0])
        else:
            a, b = line.split()
            a, b = int(a), int(b)
            
            G[-a].append(b)
            G[-b].append(a)
            
            Grev[b].append(-a)
            Grev[a].append(-b)
            
            V.add(a)
            V.add(b)
            V.add(-a)
            V.add(-b)
    
    if i != n:
        raise ValueError("Number of clauses does not match file discription.")
    
    if DEBUG > 2:
        print "\nGraph from file {0}".format(filename)
        print_defaultdict(G)
    
    return G, Grev, V

def scc_2sat(filename):
    """ Solve 2-SAT problrm from a file. """
    
    with benchmark("Read 2-SAT clauses from {0}".format(filename)) as r:
        G, Grev, V = read_2sat_clauses(filename)
    
    with benchmark("Compute SCC of {0}".format(filename)) as r:
        S = scc(G, Grev, V)
    
    with benchmark("Test SCC satisfiability of {0}".format(filename)) as r:
        for u, v_list in S.iteritems():
            if -u in v_list:
                if DEBUG > 0:
                    print "\nFailing SCC:"
                    print u, v_list, "\n"
                return 0
    
    return 1

In [5]:
DEBUG = 3

def test_scc_2sat(filename, answer):
    assert scc_2sat(filename) == answer, "SCC version of 2-SAT does not pass test case {0}.".format(filename)
    print "SCC version of 2-SAT passes test case {0}\n".format(filename)
    return

test_scc_2sat("test1.txt", 0)
test_scc_2sat("test2.txt", 1)


Graph from file test1.txt
1: [37], 5: [100, -76, -20], 6: [55], 12: [40, -27], 14: [-91, 53], 16: [-17], 17: [-16], 18: [-53, -99, 36, -75, 86, 20], 19: [32], 20: [-5], 21: [-47], 22: [-91, -75], 23: [47], 24: [52], 26: [-94], 27: [-12], 34: [71, 91, -70], 35: [54], 36: [-51, 8, 38], 38: [-54, -57, 49], 39: [76, 14], 40: [-49], 41: [1], 42: [78], 45: [89], 46: [69], 47: [-21], 48: [7, -78], 49: [-40], 51: [-36], 52: [47], 53: [-18], 54: [-38, 19], 55: [-80], 57: [-38], 64: [-97], 66: [49], 70: [-90, 67, -34], 74: [76], 75: [-18, -22], 76: [-5], 78: [-48, 44, 65], 79: [99], 80: [-55], 82: [9], 87: [13], 88: [22], 90: [-70, 91, 61], 91: [-14, -22], 92: [62], 94: [1, -26], 97: [-64], 98: [58], 99: [33, -18, 18], 100: [77], -100: [-5, 84], -99: [91, -79], -97: [87], -95: [63, 50], -91: [15, 20, -90, -34, 99, 23], -90: [80], -89: [-45, 21], -88: [22, 51], -87: [97], -86: [-18], -85: [8, 3], -84: [26, 100], -83: [81], -81: [83], -80: [90], -79: [20], -78: [-42], -77: [-100], -76: [-74, -39]

In [6]:
DEBUG = 0
results = []
for i in range(1, 7):
    name = "2sat{0}.txt".format(i)
    results.append( scc_2sat(name) )
    print

# print results

Read 2-SAT clauses from 2sat1.txt : 0.76 seconds
Compute SCC of 2sat1.txt : 1.24 seconds
Test SCC satisfiability of 2sat1.txt : 0.0805 seconds

Read 2-SAT clauses from 2sat2.txt : 2.1 seconds
Compute SCC of 2sat2.txt : 2.7 seconds
Test SCC satisfiability of 2sat2.txt : 0.00382 seconds

Read 2-SAT clauses from 2sat3.txt : 3.86 seconds
Compute SCC of 2sat3.txt : 5.67 seconds
Test SCC satisfiability of 2sat3.txt : 0.329 seconds

Read 2-SAT clauses from 2sat4.txt : 6.06 seconds
Compute SCC of 2sat4.txt : 8.57 seconds
Test SCC satisfiability of 2sat4.txt : 0.477 seconds

Read 2-SAT clauses from 2sat5.txt : 7.37 seconds
Compute SCC of 2sat5.txt : 10.5 seconds
Test SCC satisfiability of 2sat5.txt : 0.395 seconds

Read 2-SAT clauses from 2sat6.txt : 9.97 seconds
Compute SCC of 2sat6.txt : 13.9 seconds
Test SCC satisfiability of 2sat6.txt : 0.111 seconds



## 2-SAT via Papadimitriou's Local Search Algorithm

## [2-SAT via Backtracking](https://en.wikipedia.org/wiki/Backtracking#Constraint_satisfaction)
